In [2]:
import numpy as np
import pandas as pd
import csv
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.ensemble import RandomForestClassifier

In [3]:
flight_data = pd.read_csv('/home/kkanagalananthapadm1/flights.csv')
airport_data = pd.read_csv('/home/kkanagalananthapadm1/airports.csv')
holiday_data = pd.read_csv('/home/kkanagalananthapadm1/Holiday_List.csv')

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
flight_data['FLIGHT_DATE'] = pd.to_datetime(flight_data[['YEAR','MONTH','DAY']])
holiday_data['FLIGHT_DATE'] = pd.to_datetime(holiday_data[['YEAR','MONTH','DAY']])

In [134]:
# List All the redundant columns
col_list = [ 'TAIL_NUMBER','TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN','YEAR','MONTH','DAY','AIR_SYSTEM_DELAY',
            'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

In [135]:
# Drop All the redundant columns
flight_data.drop(col_list,inplace=True,axis=1)
holiday_data.drop(['YEAR','MONTH','DAY'],inplace=True,axis=1)


In [136]:
# Merge Holiday Data to Flight Data
flight_data = pd.merge(flight_data,holiday_data,on = 'FLIGHT_DATE',how='left')

# Create a bin for Target Variable Departure Delay
flight_data['DEP_DELAY_BIN'] = np.where(flight_data['DEPARTURE_DELAY'] < 15, 1, np.where(flight_data['DEPARTURE_DELAY'] < 60, 2,np.where(flight_data['DEPARTURE_DELAY'] < 180, 3,4)))


In [137]:
# Add Non-Holiday days rows with zero
flight_data['DAY_TYPE'].fillna(value=0,inplace=True)

In [138]:
# Create a bin for Target Variable Departure Delay
flight_data['DEP_DELAY_BIN'] = np.where(flight_data['DEPARTURE_DELAY'] < 15, 1, np.where(flight_data['DEPARTURE_DELAY'] < 60, 2,np.where(flight_data['DEPARTURE_DELAY'] < 180, 3,4)))


In [169]:
flight_data['ORIGIN_AIRPORT'].unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'MAF', 'DFW', 'MKE',
       'IAH', 'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'SGF', 'PHL', 'SBN',
       'RDD', 'EUG', 'IAD', 'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT',
       'SMF', 'AUS', 'MCI', 'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA',
       'SAT', 'CHS', 'SBA', 'SMX', 'IND', 'CLE', 'GSP', 'BDL', 'ABI',
       'RIC', 'BFL', 'OMA', 'RDM', 'FLL', 'CID', 'TPA', 'SYR', 'ROC',
       'TYR', 'LAN', 'XNA', 'GSO', 'EWR', 'PBI', 'RSW', 'OAK', 'PVD',
       'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'LGA', 'TUL', 'LIT', 'MSY',
       'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT', 'SAV', 'CLT', 'GRB',
       'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX', 'CWA', 'MSO', 'TTN',
       'AMA', 'CLL', 'HOU', 'JLN', 'MLI', 'RDU', 'CVG', 'MHK', 'MOB',
       'TLH', 'BHM', 'CAE', 'TXK', 'ACY', 'DTW', 'RAP', 'TUS', 'EAU',
       'DLH', 'FSD',

In [53]:
flight_data.dropna(subset=['ELAPSED_TIME','AIR_TIME'],inplace=True)

In [54]:
flight_data.shape

(5714008, 21)

In [60]:
FLIGHT_NUMBER_MEAN_DELAY_METRIC = flight_data.loc[flight_data['DEPARTURE_DELAY']>0,['AIRLINE','DEPARTURE_DELAY']].groupby(['AIRLINE']).agg(['count','mean','median','max','min'])
FLIGHT_NUMBER_MEAN_DELAY_METRIC

DEPARTURE_DELAY                               
                  count       mean median     max  min
AIRLINE                                               
AA               244539  34.225052   15.0  1988.0  1.0
AS                43358  25.852438   11.0   963.0  1.0
B6               101574  37.507600   18.0  1006.0  1.0
DL               281538  29.551439   12.0  1289.0  1.0
EV               168577  40.661970   19.0  1236.0  1.0
F9                34788  44.459268   20.0  1112.0  1.0
HA                20120  16.789314    7.0  1433.0  1.0
MQ                92822  39.890037   21.0  1544.0  1.0
NK                51922  41.832653   21.0   836.0  1.0
OO               170495  39.092126   18.0  1378.0  1.0
UA               255368  32.441054   13.0  1314.0  1.0
US                62254  28.371992   13.0   759.0  1.0
VX                23298  30.252640   13.0   644.0  1.0
WN               564396  26.847949   13.0   665.0  1.0

In [63]:
Percentage_flights_delayed = flight_data.loc[flight_data['DEPARTURE_DELAY']>0,['AIRLINE','DEPARTURE_DELAY']].groupby(['AIRLINE']).apply(lambda x: x.count())
Percentage_flights_delayed

,AIRLINE,DEPARTURE_DELAY
AIRLINE,,
AA,244539,244539
AS,43358,43358
B6,101574,101574
DL,281538,281538
EV,168577,168577
F9,34788,34788
HA,20120,20120
MQ,92822,92822
NK,51922,51922


In [65]:
# For Decision Tree Implementation

le = preprocessing.LabelEncoder()
flight_data['AIRLINE_ENCODE'] = le.fit_transform(flight_data['AIRLINE'])
flight_data['ORIGIN_AIRPORT_ENCODE'] = le.fit_transform(flight_data['ORIGIN_AIRPORT'])
flight_data['DESTINATION_AIRPORT_ENCODE'] = le.fit_transform(flight_data['DESTINATION_AIRPORT'])

In [121]:
train, test = train_test_split(flight_data, test_size = 0.25)
C = DecisionTreeClassifier(criterion='gini',splitter='best', max_depth=None, min_samples_split=1000)
Features = ['DAY_OF_WEEK','AIRLINE_ENCODE', 'ORIGIN_AIRPORT_ENCODE', 'DESTINATION_AIRPORT_ENCODE', 'DEPARTURE_TIME',
           'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DAY_TYPE']
X_train = train[Features]
y_train = train["DEP_DELAY_BIN"]
X_test = test[Features]
y_test = test["DEP_DELAY_BIN"]
dt = C.fit(X_train,y_train)
y_pred = C.predict(X_test)

In [122]:
confusion_matrix(y_test, y_pred)

array([[1154831,    9053,    2094,      90],
       [ 163026,   15007,    2581,      48],
       [  57307,    5616,    6426,     351],
       [   8791,     986,    1640,     655]])

In [123]:
accuracy_score(y_test, y_pred)

0.82388334073035951

In [124]:
precision, recall, fscore, support = score(y_test, y_pred)

In [125]:
print precision
print recall 
print fscore 
print support

[ 0.83444259  0.48943317  0.50435602  0.57255245]
[ 0.99036334  0.08306672  0.09219512  0.05425779]
[ 0.90574163  0.14202835  0.15589331  0.09912228]
[1166068  180662   69700   12072]


In [128]:
dotfile = open('/home/kkanagalananthapadm1/dtree.dat','w')
type(y_pred)
dotfile = tree.export_graphviz(dt, out_file = dotfile)

In [100]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)

In [129]:
y_pred_rf = clf.fit(X_train,y_train)

In [105]:
y_pred_rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [1]:
y_pred_final = clf.predict(X_test)

NameError: name 'clf' is not defined

In [110]:
accuracy_score(y_test, y_pred_final)

0.8567695389995954

In [112]:
rf_precision, rf_recall, rf_fscore, rf_support = score(y_test, y_pred_final)

In [113]:
print rf_precision
print rf_recall 
print rf_fscore 
print rf_support

[ 0.87774196  0.64853543  0.62791499  0.52891903]
[ 0.98035715  0.34159114  0.25943215  0.11109249]
[ 0.92621605  0.44748614  0.36716484  0.18361836]
[1165462  180839   70265   11936]
